In [1]:
import json
import re
from api.config.db_config import get_db_connection
from typing import Tuple

from ollama import Client
from ai.models.prompt.sdg_citation_prompt import sdg_citation_prompt

client = Client(host="http://127.0.0.1:11434")

conn = get_db_connection()

def get_abs(patent_number):
    # Fetch the patent data from the database
    fetch_patent_query = """
    SELECT en_abstract
    FROM patent
    WHERE patent.number = %s
    """
    cursor = conn.cursor()
    cursor.execute(fetch_patent_query, (patent_number,))
    row = cursor.fetchone()
    cursor.close()
    return row[0]


def get_description(patent_number):
    descriptions = []
    fetch_description_query = """
    SELECT description_number, description_text
    FROM patent_description
    WHERE patent_number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(fetch_description_query, (patent_number,))
    fetchall = cursor.fetchall()
    for description in fetchall:
        descriptions.append({
            "description_number": description[0],
            "description_text": description[1]
        })
    cursor.close()
    return descriptions

2025-05-27 06:16:19,217 - cep_api - DEBUG - get_db_connection - Connecting to database at localhost:5432/cep as user user


In [2]:
def extract_sdgs(text: str) -> list[str]:
        """
        Extracts and standardizes SDG references from a given text.

        This method identifies SDG mentions in various formats, including:
        - "SDG" followed by a number (e.g., "SDG1", "SDG 2").
        - Numbers with sub-targets (e.g., "16.1", "3.4"), where the main number
          is extracted.
        - Standalone numbers (1-17) that appear at the beginning of the text
          or are preceded by common delimiters (commas, semicolons, colons, whitespace)
          and followed by delimiters or the end of the string.
        The matching is case-insensitive.

        Args:
            text (str): The input text to scan for SDG references.

        Returns:
            List[str]: A list of unique SDGs found, formatted as "SDG<number>"
                (e.g., ["SDG1", "SDG2"]), sorted numerically. Returns ["None"]
                if no valid SDGs (1-17) are found or if the input text is empty
                or not a string.
        """
        if not text or not isinstance(text, str):
            # Modified to return ["None"] as per original logic for empty/invalid text
            return ["None"]

        sdg_numbers = set()  # Use set to avoid duplicates

        # Pattern 1: SDG followed by number with optional sub-target
        # Captures: SDG1, sdg 2, SDG13.4, etc.
        sdg_pattern = r'(?i)\bsdg\s*(\d{1,2})(?:\.\d+)?\b'
        sdg_matches = re.findall(sdg_pattern, text)
        for match in sdg_matches:
            number = int(match)
            if 1 <= number <= 17:
                sdg_numbers.add(number)

        # Pattern 2: Number with sub-target (e.g., "16.1", "3.4")
        # Look for patterns like X.Y where X is 1-17
        number_with_sub_pattern = r'\b(\d{1,2})\.\d+\b'
        sub_matches = re.findall(number_with_sub_pattern, text)
        for match in sub_matches:
            number = int(match)
            if 1 <= number <= 17:
                sdg_numbers.add(number)

        # Pattern 3: Standalone numbers at beginning or after delimiters
        standalone_pattern = r'(?:^|[,;:]\s*|(?<=\s))(\d{1,2})(?=\s*[,;]|\s*$|\s+)'
        standalone_matches = re.findall(standalone_pattern, text.strip())
        for match in standalone_matches:
            number = int(match)
            if 1 <= number <= 17:
                sdg_numbers.add(number)

        # Convert to sorted list of formatted strings
        result = [f"SDG{num}" for num in sorted(sdg_numbers)]

        return ["None"] if not result else result

In [9]:
import re
from typing import Tuple

def get_citation_explanation(text: str) -> Tuple[str, str]:
    """
    Extracts <summary> elements and <citation><explanation> pairs from text.
    
    The method assumes the input text contains XML-like tags `<citation>`
    and `<explanation>` delimiting the relevant content. It also expects
    a `</think>` tag preceding the main content.

    Args:
        text: The text to analyze
        
    Returns:
        Tuple containing (summary_content, formatted_citations_explanations)
    """
    # Remove the part before </think> if it exists
    if "</think>" in text:
        text = text.split("</think>", 1)[1]
    
    # Extract summary if it exists
    summary_match = re.search(r'<summary>\s*(.*?)\s*</summary>', text, re.DOTALL)
    summary_content = summary_match.group(1).strip() if summary_match else ""
    
    # Extract all citation/explanation pairs
    citation_explanation_pairs = []
    
    # Find all citations
    citation_matches = re.finditer(
        r'<citation>\s*(.*?)\s*<\/citation>', text, re.DOTALL | re.IGNORECASE
    )

    for citation_match in citation_matches:
        citation_content = citation_match.group(1).strip()
        citation_end = citation_match.end()

        # Find the corresponding explanation after the citation
        remaining_text = text[citation_end:]
        explanation_match = re.search(
            r'^\s*<explanation>\s*(.*?)\s*<\/explanation>', remaining_text, re.DOTALL | re.IGNORECASE
        )

        if explanation_match:
            explanation_content = explanation_match.group(1).strip()
            citation_explanation_pairs.append(
                (citation_content, explanation_content)
            )
    
    # Format the citation/explanation pairs
    formatted_pairs = []
    for i, (citation, explanation) in enumerate(citation_explanation_pairs, 1):
        formatted_pair = f"**Citation {i}:**\n{citation}\n**Explanation {i}:**\n{explanation}"
        formatted_pairs.append(formatted_pair)
    
    formatted_citations_explanations = "\n\n".join(formatted_pairs)
    
    return summary_content, formatted_citations_explanations



# Example usage with your text
def test_function():
    sample_text = """<think>
    Okay, let's tackle this query. The user wants an analysis linking the provided text to SDG 3: Good Health and Well-being. First, I need to understand what the text is about. It's a technical description of a medical device component, specifically a connector mechanism in a surgical stapling instrument. The key points are about ensuring proper electrical connections between the jaw member and the cartridge assembly to provide specifications like staple size, which is crucial for the device's functionality during surgery.

    Now, how does this relate to SDG 3? SDG 3 focuses on ensuring healthy lives and promoting well-being. The text's relevance here is that the device is used in medical procedures, which directly impact patient health outcomes. By ensuring the device works correctly, it contributes to safer surgeries, reducing complications and improving patient recovery. The connector mechanism's reliability is essential for accurate information transfer, which is part of the device's safety and effectiveness.

    Looking at the citations, I need to find parts of the text that mention the device's role in medical procedures. For example, in [description number 16], it says the connector mechanism provides specifications to the handle assembly, which affects the surgery's outcome. Another part in [description number 20] talks about the connector members being beveled for proper engagement, ensuring the device functions correctly during use. These details are crucial for the device's reliability, which in turn supports good health outcomes.

    I need to make sure not to mention other SDGs. Even though the device might relate to innovation or industry (SDG 9), the focus here is strictly on health. The summary should clearly state that the device's proper functioning contributes to safe and effective medical procedures, thus aligning with SDG 3. The citations must be exact and linked to the health aspect, not other areas.
    </think>

    <summary>This text is linked to SDG 3: Good Health and Well-being because it describes a medical device component (a surgical stapling instrument) designed to improve the accuracy and safety of surgical procedures. By ensuring reliable electrical connections between the device's parts, the mechanism enables precise communication of critical cartridge specifications (e.g., staple size, configuration) to the handle assembly, which directly impacts the quality of patient care during surgery. This contributes to safer, more effective medical interventions, aligning with the goal of promoting health and well-being through technological innovation in healthcare.</summary>

    <citation>The tool assembly 104 of the loading unit 100 includes a connector mechanism 130 (FIG. 16) having a first or male connector assembly 140 disposed within the jaw member 110 (FIG. 5) and a second or female connector assembly 160 (FIG. 6) secured to the cartridge assembly 120. As will be described in further detail below, when the cartridge assembly 120 is loaded into the jaw member 110 of the loading unit 100, the second connector assembly 160 of the connector mechanism 130 electrically couples with the first connector assembly 140 of the connector mechanism 130 to provide specifications of the cartridge assembly 120 to the handle assembly 12 (FIG. 1) of the surgical stapling instrument 10, e.g., staple size, number and/or configuration.</citation>  
    <explanation>Directly links the device's functionality to medical procedures by ensuring accurate transmission of critical surgical parameters (e.g., staple size), which enhances the safety and effectiveness of surgeries, a core aspect of SDG 3's focus on health outcomes.</explanation>  

    <citation>First and second connector members 146, 148 extend outwardly from the first connector housing 142 and are positioned and configured to engage the second connector assembly 160 (FIG. 6) of the connector mechanism 130 when the cartridge assembly 120 is received within the channel 111 of the jaw member 110.</citation>  
    <explanation>Highlights the mechanical precision required for the device to function correctly during surgery, ensuring reliable performance that supports safe medical interventions and aligns with SDG 3's objective of promoting health through technological reliability.</explanation>
    """
    
    summary, citations_explanations = get_citation_explanation(sample_text)
    
    print("SUMMARY:")
    print(summary)
    print("\n" + "="*50 + "\n")
    print("CITATIONS AND EXPLANATIONS:")
    print(citations_explanations)

test_function()

SUMMARY:
This text is linked to SDG 3: Good Health and Well-being because it describes a medical device component (a surgical stapling instrument) designed to improve the accuracy and safety of surgical procedures. By ensuring reliable electrical connections between the device's parts, the mechanism enables precise communication of critical cartridge specifications (e.g., staple size, configuration) to the handle assembly, which directly impacts the quality of patient care during surgery. This contributes to safer, more effective medical interventions, aligning with the goal of promoting health and well-being through technological innovation in healthcare.


CITATIONS AND EXPLANATIONS:
**Citation 1:**
The tool assembly 104 of the loading unit 100 includes a connector mechanism 130 (FIG. 16) having a first or male connector assembly 140 disposed within the jaw member 110 (FIG. 5) and a second or female connector assembly 160 (FIG. 6) secured to the cartridge assembly 120. As will be des

In [8]:
def create_sdg_summary(sdg_summary: dict):
    """
    Insert SDG summary data into the PostgreSQL database.

    Args:
        sdg_summary (dict): A dictionary containing SDG summary data. Keys should include:
            - patent_number (str): The patent number.
            - sdg (str): The SDG related to the patent.
            - sdg_reason (str): The reason why the patent is related to the SDG.
            - sdg_details (str): The text related to the SDG in the patent.

    Returns:
        None
    """
    conn = get_db_connection()
    cursor = conn.cursor()

    # Insert SDG summary data into the sdg_summary table
    insert_sdg_summary_query = """
    INSERT INTO patent_sdg_summary (patent_number, sdg, sdg_reason, sdg_details)
    VALUES (%s, %s, %s, %s)
    ON CONFLICT (patent_number, sdg) DO NOTHING;
    """

    cursor.execute(insert_sdg_summary_query, (
        sdg_summary["patent_number"],
        sdg_summary["sdg"],
        sdg_summary["sdg_reason"],
        sdg_summary["sdg_details"]
    ))

    conn.commit()
    cursor.close()

    # Set is_analyzed to True for the patent in the patents table
    update_patent_query = """
    UPDATE patent
    SET is_analyzed = TRUE
    WHERE number = %s;
    """
    cursor = conn.cursor()
    cursor.execute(update_patent_query, (sdg_summary["patent_number"],))
    conn.commit()
    cursor.close()

    # Close the database connection
    conn.close()

In [11]:
with open("../src/ai/testsets/dataset_labeled.jsonl", "r", encoding="utf-8") as f:
    with open("ouput_data.jsonl", "w", encoding="utf-8") as f_out:
        for line in f:
            iteam = json.loads(line)
            for key, value in iteam.items():
                if key != "reason":
                    patent_number = key
                    abs = get_abs(patent_number)
                    sdgs = extract_sdgs(value)
                    descriptions = get_description(patent_number)
                    patent_text = f"[abstract]: {abs}\n"
                    for description in descriptions:
                        patent_text += f"[description number {description["description_number"]}]: {description["description_text"]}\n"
                else:
                    reason = value
            
            patent_text = " ".join(patent_text.split()[:3000])

            for sdg in sdgs:
                if sdg != "None":
                    formatted_prompt = sdg_citation_prompt(patent_text, sdg)

                    response_data = client.generate(
                        model="qwen3:14b", # Make sure this model is available in your Ollama instance
                        prompt=formatted_prompt,
                        options={"temperature": 0.2}
                    )
                    raw_output_text = response_data.get('response', '').strip()
                    summary_content, formatted_citations_explanations = get_citation_explanation(raw_output_text)

                    print(raw_output_text)

                # If not a SDG the previous model already made an explanation
                else:
                    summary_content, formatted_citations_explanations = reason, ""

                sdg_summary_detail = {
                    "patent_number": patent_number,
                    "sdg": sdg,
                    "sdg_reason": summary_content,
                    "sdg_details": formatted_citations_explanations.replace("\n", "\\n")
                }
                
                create_sdg_summary(sdg_summary_detail)
                
                # Save
                f_out.write(json.dumps(sdg_summary_detail, ensure_ascii=False) + "\n")
                f_out.flush()

2025-05-27 06:33:25,215 - httpcore.connection - DEBUG - trace - connect_tcp.started host='127.0.0.1' port=11434 local_address=None timeout=None socket_options=None
2025-05-27 06:33:25,216 - httpcore.connection - DEBUG - trace - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7b5049ba63f0>
2025-05-27 06:33:25,216 - httpcore.http11 - DEBUG - trace - send_request_headers.started request=<Request [b'POST']>
2025-05-27 06:33:25,217 - httpcore.http11 - DEBUG - trace - send_request_headers.complete
2025-05-27 06:33:25,217 - httpcore.http11 - DEBUG - trace - send_request_body.started request=<Request [b'POST']>
2025-05-27 06:33:25,217 - httpcore.http11 - DEBUG - trace - send_request_body.complete
2025-05-27 06:33:25,217 - httpcore.http11 - DEBUG - trace - receive_response_headers.started request=<Request [b'POST']>
2025-05-27 06:34:04,509 - httpcore.http11 - DEBUG - trace - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Ty

<think>
Okay, let's tackle this query. The user wants an analysis linking the provided text to SDG 9. First, I need to understand what SDG 9 entails. It's about industry, innovation, and infrastructure—building resilient infrastructure, promoting sustainable industrialization, and fostering innovation.

Looking at the text, it's about a sensor used in machinery like construction equipment. The sensor measures linear positions of mechanical components, which is part of industrial machinery. The innovation here is the design of the sensor, which includes features like a removable cover for maintenance and drainage passages to prevent blockages. These features enhance the durability and functionality of the infrastructure (machinery) it's part of.

Now, the user wants a structured analysis with two parts. Part 1 is a summary explaining the link to SDG 9. I need to focus solely on SDG 9. The key points here are innovation in the sensor design, promoting sustainable industrialization throug

2025-05-27 06:34:45,399 - httpcore.http11 - DEBUG - trace - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 27 May 2025 04:34:45 GMT'), (b'Transfer-Encoding', b'chunked')])
2025-05-27 06:34:45,399 - httpx - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-05-27 06:34:45,400 - httpcore.http11 - DEBUG - trace - receive_response_body.started request=<Request [b'POST']>
2025-05-27 06:34:45,400 - httpcore.http11 - DEBUG - trace - receive_response_body.complete
2025-05-27 06:34:45,400 - httpcore.http11 - DEBUG - trace - response_closed.started
2025-05-27 06:34:45,400 - httpcore.http11 - DEBUG - trace - response_closed.complete
2025-05-27 06:34:45,402 - cep_api - DEBUG - get_db_connection - Connecting to database at localhost:5432/cep as user user
2025-05-27 06:34:45,479 - httpcore.http11 - DEBUG - trace - send_request_headers.started req

<think>
Okay, let's tackle this query. The user wants an analysis linking the provided text to SDG 9: Industry, Innovation and Infrastructure. First, I need to understand the content of the text.

The text discusses an aluminum alloy manufacturing process involving TiB2 aggregates and crystal-grain refining agents. It mentions improving industrial processes through innovation in materials science, which could relate to SDG 9's focus on fostering innovation and sustainable industrialization.

Now, I need to ensure that the summary and citations exclusively focus on SDG 9. Let me check the text again for relevant parts. The text talks about enhancing the efficiency of the manufacturing process by reducing the amount of refining agent needed, which could be part of sustainable industrial practices. Also, the development of new materials and techniques would fall under innovation.

Looking at the citations, I should find excerpts that mention innovation, industrial processes, or infrastruc

2025-05-27 06:35:20,312 - httpcore.http11 - DEBUG - trace - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 27 May 2025 04:35:20 GMT'), (b'Transfer-Encoding', b'chunked')])
2025-05-27 06:35:20,313 - httpx - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-05-27 06:35:20,313 - httpcore.http11 - DEBUG - trace - receive_response_body.started request=<Request [b'POST']>
2025-05-27 06:35:20,313 - httpcore.http11 - DEBUG - trace - receive_response_body.complete
2025-05-27 06:35:20,314 - httpcore.http11 - DEBUG - trace - response_closed.started
2025-05-27 06:35:20,314 - httpcore.http11 - DEBUG - trace - response_closed.complete
2025-05-27 06:35:20,315 - cep_api - DEBUG - get_db_connection - Connecting to database at localhost:5432/cep as user user
2025-05-27 06:35:20,426 - httpcore.http11 - DEBUG - trace - send_request_headers.started req

...

<think>
Okay, let's tackle this query. The user wants an analysis linking the provided text to SDG 3: Good Health and Well-being. First, I need to understand the content of the text. It's about a new anesthetic composition using hyaluronic acid hydrogels and lipid matrix particles, designed for long-acting local anesthesia. The goal is to reduce the need for opioids, which can have side effects, thereby improving patient outcomes.

Now, the main point is to connect this innovation to SDG 3. The key here is that the text discusses a medical advancement that promotes health by providing a safer, more effective anesthetic. This directly relates to ensuring healthy lives and well-being. I need to make sure not to mention other SDGs like SDG 10 or 12, even if they might be tangentially related.

For Part 1, the summary should clearly state that the text is linked to SDG 3 because it presents a medical innovation that improves health outcomes by reducing opioid use and enhancing anesthetic e

2025-05-27 12:09:49,306 - httpcore.http11 - DEBUG - trace - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 27 May 2025 10:09:49 GMT'), (b'Transfer-Encoding', b'chunked')])
2025-05-27 12:09:49,306 - httpx - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-05-27 12:09:49,307 - httpcore.http11 - DEBUG - trace - receive_response_body.started request=<Request [b'POST']>
2025-05-27 12:09:49,307 - httpcore.http11 - DEBUG - trace - receive_response_body.complete
2025-05-27 12:09:49,307 - httpcore.http11 - DEBUG - trace - response_closed.started
2025-05-27 12:09:49,307 - httpcore.http11 - DEBUG - trace - response_closed.complete
2025-05-27 12:09:49,309 - cep_api - DEBUG - get_db_connection - Connecting to database at localhost:5432/cep as user user
2025-05-27 12:09:49,392 - httpcore.http11 - DEBUG - trace - send_request_headers.started req

<think>
Okay, let's tackle this query. The user wants a structured analysis linking the provided text to SDG 9. First, I need to understand why the text is relevant to SDG 9. The text discusses a CHQ wire rod and processed product with high strength, cold forging characteristics, and resistance to hydrogen delayed fracture. These materials are used in automotive parts, which relates to industrial innovation and infrastructure.

SDG 9 focuses on industry, innovation, and infrastructure. The development of advanced materials like the CHQ steel mentioned here supports sustainable industrialization by improving product durability and performance. The innovation in material science contributes to building resilient infrastructure, as automotive parts need to be reliable and long-lasting. Also, the mention of fostering innovation through the tempering process and alloy composition aligns with SDG 9's goal of promoting innovation.

Now, looking at the citations. The text mentions the processe

2025-05-27 12:10:17,114 - httpcore.http11 - DEBUG - trace - receive_response_headers.failed exception=KeyboardInterrupt()
2025-05-27 12:10:17,114 - httpcore.http11 - DEBUG - trace - response_closed.started
2025-05-27 12:10:17,115 - httpcore.http11 - DEBUG - trace - response_closed.complete


KeyboardInterrupt: 

## Test with class in production

In [ ]:
# from ai.models.CitationPatent import CitationPatent
# from api.config.ai_config import ai_client, ai_model

# citationPatent = CitationPatent(client=ai_client, model_name=ai_model)

# with open("../src/ai/testsets/dataset_labeled.jsonl", "r", encoding="utf-8") as f:
#     with open("ouput_data.jsonl", "w", encoding="utf-8") as f_out:
#         for line in f:
#             iteam = json.loads(line)
#             for key, value in iteam.items():
#                 if key != "reason":
#                     patent_number = key
#                     abs = get_abs(patent_number)
#                     sdgs = extract_sdgs(value)
#                     descriptions = get_description(patent_number)
#                     patent_text = f"[abstract]: {abs}\n"
#                     for description in descriptions:
#                         patent_text += f"[description number {description["description_number"]}]: {description["description_text"]}\n"
#                 else:
#                     reason = value
            
#             patent_text = " ".join(patent_text.split()[:3000])

#             for sdg in sdgs:
                
#                 citation_content, explanation_content = citationPatent.citation(patent_text, sdg, reason)

#                 sdg_summary_detail = {
#                     "patent_number": patent_number,
#                     "sdg": sdg,
#                     "sdg_reason": explanation_content,
#                     "sdg_details": citation_content
#                 }

#                 print(sdg_summary_detail)
            
#                 # Save
#                 f_out.write(json.dumps(sdg_summary_detail, ensure_ascii=False) + "\n")
#                 f_out.flush()

2025-05-27 05:08:44,243 - httpx - INFO - _send_single_request - HTTP Request: GET http://localhost:11434/api/tags "HTTP/1.1 200 OK"
2025-05-27 05:08:59,537 - httpx - INFO - _send_single_request - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-05-27 05:08:59,539 - cep_api - WARNING - generate_response - Unexpected output type from LLM client: <class 'ollama._types.GenerateResponse'>. Converting to string.


{'patent_number': 'EP4191828A1', 'sdg': 'None', 'sdg_reason': 'This passage describes the technical components of the device, focusing on hardware features without mentioning any SDG-related objectives.', 'sdg_details': 'According to an embodiment, the electronic device 100 may include at least one or more of a display 101, audio modules 103 and 104, camera modules 105, 113, 114, and 115, a key input device 107, and a connector hole 106.'}


2025-05-27 05:09:10,016 - httpx - INFO - _send_single_request - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-05-27 05:09:10,017 - cep_api - WARNING - generate_response - Unexpected output type from LLM client: <class 'ollama._types.GenerateResponse'>. Converting to string.


{'patent_number': 'EP4140416A1', 'sdg': 'SDG3', 'sdg_reason': 'The connector mechanism electrically couples with the first connector assembly to provide specifications of the cartridge (e.g., staple size, number, and configuration) to the handle assembly, ensuring accurate medical procedures and patient safety.', 'sdg_details': '[description number 16]'}
